In [ ]:
from quantopian.research.experimental import continuous_future, history
import matplotlib.pyplot as plt


months = list("FGHJKMNQUVXZ")
dict_month = dict(zip(range( 1,13), months))
dict_month


In [ ]:
# ガソリンの2017年12月限をショート，2018年12月限ロング
symbol = "SY"
target_month = 1
month = dict_month[target_month]
year1 = 16
year2 = 17

f1 = symbols("{}{}{}".format(symbol ,month, year1)) 
f2 = symbols("{}{}{}".format(symbol, month, year2)) 

start = f2.start_date # f2 のスタート日から
end = f1.expiration_date # f1 の満期日までのデータを取得

pan = history(
 [f1, f2], 
 fields=["price", "volume"],
 frequency='daily',
 start=start,
 end=end
 )

In [ ]:
f1.to_dict()

In [ ]:
df = pan.price
df_change = df.pct_change()
(df_change[f2] - df_change[f1]).cumsum().plot(title="spread")
pan.volume.plot(title="volume")

In [ ]:
months = list("FGHJKMNQUVXZ")
list(zip(range( 1,13), months))
symbol = "XB"
years = [15,16,17,18]

syms = list()
for year in years:
    for month in months:
        syms.append("{}{}{}".format(symbol ,month, year))
        
        
## 一つ前の限月が終わった日からデータを取得するために，
## [一つ前の限月，対象の限月，一年後の限月]というリストを作りデータ取得の時に使う．
sym_group = list()
for i in range(len(syms)):
    try:
        sym_group.append([symbols(syms[i]), symbols(syms[i+1]), symbols(syms[i+13])])
    except:
        pass 
   


In [ ]:
results = dict()

# ココで，f_preは一つ前の限月
# f1,f2は対象の限月，一年後の限月
# 例；f_pre：2017年1月限，f1: 2017年2月限，f2: 2018年2月限
# 2017年1月限が満期日を迎えた日から，2017年2月限が満期日を迎える日までの f1とf2のヒストリカルデータ
# を取得する．

for pre_symbol, target_symbol, next_symbol in sym_group: 
    f_pre, f1, f2 = pre_symbol, target_symbol, next_symbol
    start = f_pre.expiration_date
    end = f1.expiration_date
    df = history([f1, f2], 
                 fields="price",
                 frequency='daily',
                 start=start,
                 end=end)
    #df_change = df.pct_change()
    #results[f1.symbol] = (df_change[f2] - df_change[f1]).cumsum().values
    results[f1.symbol] = df

    
    

In [ ]:
def draw(result_dict):
    contango_list = list()
    profit_list = list()
    
    for k, df in result_dict.items():
        # 期近のseries
        near_position = df.iloc[:, 0]
        # 1年限のseries
        far_position = df.iloc[:, 1]
        # データ取得初日の期近と1年限の比率を取得する．
        contango = far_position[0] / near_position[0] - 1
        # 初日に期近をショート，1年後をロングした場合，データ最終日のPL
        profit = near_position[0] - near_position[-1] + far_position[-1] - near_position[0]
        contango_list.append(contango)
        profit_list.append(profit)
        
    # データ初日の期近と1年限の比率と，そのロングショートのPLの関係を散布図に描画
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.axvline(color='red')
    ax.axhline(color='red')
    ax.scatter(x=contango_list, y=profit_list, marker = 'o', )


draw(results)